In [1]:
import math
import numpy as np
import pandas as pd
import random
import xgboost as xgb
from sklearn import preprocessing, model_selection
from sklearn.metrics import log_loss
import string
from sklearn.feature_extraction.text import  CountVectorizer
from scipy.stats import boxcox
from scipy import stats
from scipy import sparse

y_map = {'low': 2, 'medium': 1, 'high': 0}

train = pd.read_json('data/backup/train.json')
test = pd.read_json('data/backup/test.json')

y_train = train['interest_level'].apply(lambda x: y_map[x]).reset_index(drop=True)
train = train.drop(['listing_id', 'interest_level'], axis=1)
listing_id = test.listing_id
test = test.drop('listing_id', axis=1)

y_train.to_json('data/train_interest.json', orient='records')
listing_id.to_json('data/test_ids.json', orient='records')

/Users/jgzuke/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
ntrain = train.shape[0]
train_test = pd.concat((train, test), axis=0).reset_index(drop=True)

# convert date to be more useable
train_test['Date'] = pd.to_datetime(train_test['created'])
# year is all the same
#train_test['Year'] = train_test['Date'].dt.year
train_test['Month'] = train_test['Date'].dt.month
train_test['Day'] = train_test['Date'].dt.day
train_test['Wday'] = train_test['Date'].dt.dayofweek
train_test['Yday'] = train_test['Date'].dt.dayofyear
train_test['hour'] = train_test['Date'].dt.hour
train_test = train_test.drop(['Date', 'created'], axis=1)


# check if this building id/manager id and add worths is 0
train_test['Zero_building_id'] = train_test['building_id'].apply(lambda x: 1 if x == '0' else 0)
train_test['Zero_manager_id'] = train_test['manager_id'].apply(lambda x: 1 if x == '0' else 0)


# mess with description meta data
train_test['desc'] = train_test['description']
train_test['desc'] = train_test['desc'].apply(lambda x: x.replace('<p><a  website_redacted ', ''))
train_test['desc'] = train_test['desc'].apply(lambda x: x.replace('!<br /><br />', ''))

string.punctuation.__add__('!!')
string.punctuation.__add__('(')
string.punctuation.__add__(')')
remove_punct_map = dict.fromkeys(map(ord, string.punctuation))

train_test['desc'] = train_test['desc'].apply(lambda x: x.translate(remove_punct_map))
train_test['desc_letters_count'] = train_test['description'].apply(lambda x: len(x.strip()))
train_test['desc_words_count'] = train_test['desc'].apply(lambda x: 0 if len(x.strip()) == 0 else len(x.split(' ')))
train_test['desc_words_length'] = (train_test['desc_letters_count'] / train_test['desc_words_count']).apply(lambda x: 0 if math.isnan(x) or math.isinf(x) else x)

train_test.drop(['description', 'desc'], axis=1, inplace=True)


# address
train_test['address1'] = train_test['display_address']
train_test['address1'] = train_test['address1'].apply(lambda x: x.lower())

address_map = {
    'w': 'west',
    'st.': 'street',
    'ave': 'avenue',
    'st': 'street',
    'e': 'east',
    'n': 'north',
    's': 'south'
}

def address_map_func(s):
    s = s.split(' ')
    out = []
    for x in s:
        if x in address_map:
            out.append(address_map[x])
        else:
            out.append(x)
    return ' '.join(out)


train_test['address1'] = train_test['address1'].apply(lambda x: x.translate(remove_punct_map))
train_test['address1'] = train_test['address1'].apply(lambda x: address_map_func(x))

new_cols = ['street', 'avenue', 'east', 'west', 'north', 'south']
for col in new_cols:
    train_test[col] = train_test['address1'].apply(lambda x: 1 if col in x else 0)

train_test['other_address'] = train_test[new_cols].apply(lambda x: 1 if x.sum() == 0 else 0, axis=1)
train_test.drop(['display_address', 'street_address'], axis=1, inplace=True)


# features (better not to lower)
train_test['features_count'] = train_test['features'].apply(lambda x: len(x))
train_test['features2'] = train_test['features']
train_test['features2'] = train_test['features2'].apply(lambda x: ' '.join([''.join(i.replace('_',' ').replace('-',' ').split(' ')) for i in x]))

c_vect = CountVectorizer(stop_words='english', max_features=200, ngram_range=(1, 1))
c_vect.fit(train_test['features2'])

c_vect_sparse_1 = c_vect.transform(train_test['features2'])
c_vect_sparse1_cols = c_vect.get_feature_names()
train_test.drop(['features', 'features2'], axis=1, inplace=True)


# photos
train_test['photos_count'] = train_test['photos'].apply(lambda x: len(x))
train_test.drop(['photos'], axis=1, inplace=True)


# convert ['building_id', 'manager_id', 'address1'] to enumerated labels
categoricals = [x for x in train_test.columns if train_test[x].dtype == 'object']
for feat in categoricals:
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(train_test[feat].values))
    train_test[feat] = lbl.transform(list(train_test[feat].values))


# transform managers and building ids
train_test['manager_id'] = train_test['manager_id'].apply(lambda x: str(x))
train_test['manager_id'], labels = pd.factorize(train_test['manager_id'].values, sort=True)
train_test['building_id'] = train_test['building_id'].apply(lambda x: str(x))
train_test['building_id'], labels = pd.factorize(train_test['building_id'].values, sort=True)


# transform price
bc_price, tmp = boxcox(train_test.price)
train_test['bc_price'] = bc_price


# add price per bed/bath
train_test['price_per_bedroom'] = train_test.apply(lambda x: x.price / max(1, x.bedrooms), axis=1)
train_test['price_per_bathroom'] = train_test.apply(lambda x: x.price / max(1, x.bathrooms), axis=1)
train_test['price_per_bed_and_bath'] = train_test.apply(lambda x: x.price / (max(1, x.bedrooms) + max(1, x.bathrooms)*0.1), axis=1)


# add sparse
train_test_cv1_sparse = sparse.hstack((train_test, c_vect_sparse_1)).tocsr()
x_train = train_test_cv1_sparse[:ntrain, :]
x_test = train_test_cv1_sparse[ntrain:, :]

# add feature names
features = list(train_test.columns) + ['sparse_' + vect_name for vect_name in c_vect_sparse1_cols]
dtrain_data = pd.DataFrame(np.array(x_train.todense()), columns=features)
dtest_data = pd.DataFrame(np.array(x_test.todense()), columns=features)

In [3]:
dtrain_data.to_json('data/train.json', orient='records')
dtest_data.to_json('data/test.json', orient='records')

In [4]:
test = pd.read_json('data/backup/test.json')
test2 = pd.read_json('data/test.json')
c = pd.read_json('data/test_ids.json', typ='series')

In [5]:
c[:10]

,0
0,7142618
1,7210040
2,6832604
3,6830595
4,6843709
5,7232076
6,6832266
7,6931714
8,7316357
9,6839137


In [8]:
test[:10][['latitude', 'listing_id']]

,latitude,listing_id
0,40.7185,7142618
1,40.7278,7210040
2,40.7330,6832604
3,40.6652,6830595
4,40.7807,6843709
5,40.7784,7232076
6,40.6980,6832266
7,40.7656,6931714
8,40.7519,7316357
9,40.7631,6839137


In [7]:
test2[:10][['latitude']]

,latitude
0,40.7185
1,40.7278
2,40.7330
3,40.6652
4,40.7807
5,40.7784
6,40.6980
7,40.7656
8,40.7519
9,40.7631
